In [1]:
import csv
import pandas as pd
import numpy as np
import cvxpy as cp
import time
from scipy.stats import chi2
from itertools import product
from scipy.linalg import block_diag
from scipy.optimize import minimize

In [2]:
df = pd.read_csv("E:/RA/RA for Lorenzo/Data/Processed/airlinedata.csv")

In [3]:
# data cleaning
df['non_hub_entry'] = df['non_hub_major_entry'] | df['non_major_entry']
df['mktpres_non_hub_bi'] = df['mktpres_non_hub_major_bi'] | df['mktpres_non_major_bi']
df['y'] = list( zip(df['hub_airline_entry'], df['non_hub_entry']) )
df['Z'] = list( zip(df['marketsize_bi'], df['mktpres_hub_airline_bi'], df['mktpres_non_hub_bi']) )
df = df.drop(['marketsize_bi', 'mktpres_hub_airline_bi', 'mktpres_non_hub_major_bi', 'mktpres_non_major_bi', 'hub_airline_entry', 'non_hub_major_entry', 'non_major_entry', 'non_hub_entry', 'mktpres_non_hub_bi' ], axis=1)

In [4]:
n = len(df)
df # data structure

,id,y,Z
0,1,"(0, 1)","(0, 0, 1)"
1,2,"(1, 1)","(0, 1, 1)"
2,3,"(1, 1)","(0, 1, 0)"
3,4,"(0, 1)","(1, 0, 1)"
4,5,"(0, 1)","(1, 1, 1)"
...,...,...,...
640,641,"(1, 0)","(1, 1, 1)"
641,642,"(0, 1)","(0, 0, 0)"
642,643,"(1, 0)","(0, 1, 0)"
643,644,"(0, 1)","(0, 1, 0)"


In [5]:
y_list = list(product([0, 1], repeat=2))
Z_list = list(product([0, 1], repeat=3)) 
m = []

In [6]:
y_list # 4 potential equilibria

[(0, 0), (0, 1), (1, 0), (1, 1)]

In [7]:
Z_list # 8 market characteristics types

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [8]:
# choice probability vector 
# this contains many 0 since not every equilibrium is observed
m = []
for Zj in Z_list:
    filtered_df = df[df['Z'] == Zj]
    if len(filtered_df) == 0:  # Skip if no rows match the current Zj
        m.extend([0] * len(y_list))
        continue
    for yi in y_list:
        count_y = len(filtered_df[filtered_df['y'] == yi])
        fraction = count_y / len(filtered_df)
        m.append(fraction)
        
print(len(m))
m

32


[0.0,
 0.9130434782608695,
 0.043478260869565216,
 0.043478260869565216,
 0.008771929824561403,
 0.8771929824561403,
 0.0,
 0.11403508771929824,
 0.0,
 0.14925373134328357,
 0.3582089552238806,
 0.4925373134328358,
 0.0,
 0.19327731092436976,
 0.23529411764705882,
 0.5714285714285714,
 0.0,
 0.8888888888888888,
 0.0,
 0.1111111111111111,
 0.0,
 0.625,
 0.0,
 0.375,
 0.0,
 0.13043478260869565,
 0.2608695652173913,
 0.6086956521739131,
 0.0,
 0.1484375,
 0.1953125,
 0.65625]

In [9]:
# print existing equilbira for each market characteristic type
def find_nonzero_moments(m):
    sublists = []
    for i in range(0, len(m), 4):
        sublist = [j-i for j in range(i, i + 4) if m[j] > 0.01 ]
        sublists.append(sublist)

    return sublists

non_zero_moments = find_nonzero_moments(m)
converted_list = [[y_list[i] for i in sublist] for sublist in non_zero_moments]

# Print result
for sublist in converted_list:
    print(sublist)

[(0, 1), (1, 0), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 0), (1, 1)]
[(0, 1), (1, 0), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 1)]
[(0, 1), (1, 0), (1, 1)]
[(0, 1), (1, 0), (1, 1)]


In [10]:
# construct vector "moment" for test
# this redefined vector has no zero probabilities

moments = []

# Loop over each Z in Z_list
for Zj, equilibria in zip(Z_list, non_zero_moments):
    
    # Removing the last equilibrium to avoid linear dependence
    equilibria = equilibria[:-1]
    
    # Filter the dataframe for the current Z
    filtered_df = df[df['Z'] == Zj]
    
    for yi in equilibria:
        # Convert yi back to the equilibrium tuple for comparison
        y_tuple = y_list[yi]
        
        count_y = len(filtered_df[filtered_df['y'] == y_tuple])
        fraction = count_y / len(filtered_df) if len(filtered_df) > 0 else 0
        moments.append(fraction)
        
print(len(moments))

moments

13


[0.9130434782608695,
 0.043478260869565216,
 0.8771929824561403,
 0.14925373134328357,
 0.3582089552238806,
 0.19327731092436976,
 0.23529411764705882,
 0.8888888888888888,
 0.625,
 0.13043478260869565,
 0.2608695652173913,
 0.1484375,
 0.1953125]

In [11]:
# bootstrapping for the covariance matrix \hat\Sigma

n_iterations = 1000 # bootstrap iterations
all_m_values = []
np.random.seed(77)

for _ in range(n_iterations):
    bootstrap_sample = df.sample(n=len(df), replace=True)
    
    boot_m = []
    
    for Zj, equilibria in zip(Z_list, non_zero_moments):
    
        # Removing the last equilibrium to avoid linear dependence
        equilibria = equilibria[:-1]

        # Filter the dataframe for the current Z
        filtered_df = bootstrap_sample[bootstrap_sample['Z'] == Zj]

        for yi in equilibria:
            # Convert yi back to the equilibrium tuple for comparison
            y_tuple = y_list[yi]

            count_y = len(filtered_df[filtered_df['y'] == y_tuple])
            fraction = count_y / len(filtered_df) if len(filtered_df) > 0 else 0
            boot_m.append(fraction)
        
    all_m_values.append(boot_m)

In [12]:
# Compute the variance (covariance) matrix
m_matrix = np.array(all_m_values)
Sigma_hat = np.cov(m_matrix, rowvar=False)

In [13]:
rank_of_cov_matrix = np.linalg.matrix_rank(Sigma_hat)

if rank_of_cov_matrix == Sigma_hat.shape[0]:
    print("The covariance matrix has full rank.")
else:
    print(f"The covariance matrix does not have full rank. Its rank is {rank_of_cov_matrix}.")

The covariance matrix has full rank.


In [14]:
# compute the inverse
Sigma_hat_inverse = np.linalg.inv(Sigma_hat)
Sigma_hat_inverse

array([[ 5.47676936e+02,  5.11332586e+02,  5.11401829e+01,
         5.93672490e+00, -1.66992181e+01, -8.12972323e+00,
         7.83226143e+00, -1.13154412e+01, -1.61564180e+01,
        -4.12963764e+01,  5.60508133e+00,  2.56403478e+01,
         1.45348059e+01],
       [ 5.11332586e+02,  1.03860779e+03,  7.76838957e+01,
        -2.45862821e+00, -1.71526370e+01,  3.98136342e+01,
         1.61245336e+01,  1.74626098e+01, -2.15074114e+01,
        -3.83297624e+01, -9.40912661e-01, -1.17824798e+00,
         3.06402200e+01],
       [ 5.11401829e+01,  7.76838957e+01,  1.06097064e+03,
         1.09575408e+01,  9.61126417e-01,  4.95237479e+01,
         1.22837957e+01, -2.11550588e+01,  4.98036478e+01,
        -4.33853651e+01,  1.04483260e+00,  1.10811989e+01,
         1.22332855e+01],
       [ 5.93672490e+00, -2.45862821e+00,  1.09575408e+01,
         5.72317038e+02,  1.32388745e+02, -7.77530573e+00,
        -3.27779793e+01,  1.15777447e+01, -7.72386043e+00,
         1.58165519e+01,  2.52055673e

In [15]:
# now we construct the matrix A

def generate_submatrix(j):
    # Create the identity matrix of size (j-1)x(j-1)
    I = np.eye(j-1)
    
    # Create row vectors of 1s and -1s
    row_vector_ones = np.ones((1, j-1))
    row_vector_neg_ones = -row_vector_ones
    
    # Vertically stack the matrices and vectors to form the sub-matrix
    sub_matrix = np.vstack((I, row_vector_neg_ones, -I, row_vector_ones))
    
    return sub_matrix

sub_matrices = [generate_submatrix(len(eql)) for eql in non_zero_moments]

# Combining sub-matrices into a block diagonal matrix
A = block_diag(*sub_matrices)

print(A.shape)

(42, 13)


In [16]:
# Now we construct  𝑏(𝜃)
# 𝜃=[𝛼1,𝛼2,𝛽1,𝛽2,𝛾1,𝛾2,𝛿1,𝛿2,𝜎]

def simulate_bounds_for_selected_eql(theta, Z, s, selected_eql):
    
    # Define the utility function
    def utility(i, y, Z, epsilon):
        alpha = theta[0:2]
        beta = theta[2:4]
        gamma = np.array([theta[4], theta[5]])
        delta = np.array([theta[6], theta[7]])

        S_m = Z[0]
        X_im = Z[i + 1]

        other_players = [idx for idx in range(2) if idx != i]

        return alpha[i] * S_m + beta[i] * X_im + sum(gamma[i] * np.array([y[op] for op in other_players])) + delta[i] + epsilon

    
     # Simulation 
    H_l = np.zeros(len(selected_eql))
    H_u = np.zeros(len(selected_eql))
    
    eql_in_Z = [y_list[idx] for idx in selected_eql]
    
    np.random.seed(66)
    n = 0 # number of effective draws
    while n < s:
        epsilon_sample = np.random.normal(0, theta[-1], 2)
        possible_equilibria = []
        for y in eql_in_Z:
            if all((utility(i, y, Z, epsilon_sample[i]) >= 0) == y[i] for i in range(2)):
                possible_equilibria.append(y)
                
        if len(possible_equilibria) != 0:
            n += 1
            for y_index, y in enumerate(eql_in_Z):
                if y in possible_equilibria:
                    H_u[y_index] += 1
                    if len(possible_equilibria) == 1:
                        H_l[y_index] += 1
            
   
    H_l = H_l/n
    H_u = H_u/n
        
    
    return H_l, H_u


In [17]:
def generate_b(theta, non_zero_moments, s=100):
    Z_values = list(product([0, 1], repeat=3))
    b_vector = []
    
    for idx, Z in enumerate(Z_values):
        H_l, H_u = simulate_bounds_for_selected_eql(theta, Z, s, non_zero_moments[idx])
        H_u[-1] = H_u[-1] - 1 # H_u_j - 1, the other H_u_i i=1,...,j-1 unchanged
        b_vector.extend(H_u)  # Adding H_u values
        
        # Modify H_l values according to the given rules
        modified_H_l = [-val for val in H_l[:-1]]  # -H_l_1...-H_l_j-1
        modified_H_l.append(1 - H_l[-1])  # 1-H_l_j
        
        b_vector.extend(modified_H_l)
        
    return b_vector

In [18]:
# we test the generating function using an example

theta_example = [12, 0.6, 0.7, 1.5, 1.6, 1.2, 0.2, 0.3, 2]
s = 100

# Start the timer
start_time = time.time()

# Run the function
b_vector = generate_b(theta_example, non_zero_moments, s)

# End the timer
end_time = time.time()

# Print the elapsed time
print(f"The function took {end_time - start_time} seconds to complete.")

print(len(b_vector))
b_vector

The function took 0.06669044494628906 seconds to complete.
42


[0.12,
 0.11,
 -0.22999999999999998,
 -0.12,
 -0.11,
 0.22999999999999998,
 0.16,
 -0.16000000000000003,
 -0.16,
 0.16000000000000003,
 0.07,
 0.12,
 -0.18999999999999995,
 -0.07,
 -0.12,
 0.18999999999999995,
 0.1,
 0.02,
 -0.12,
 -0.1,
 -0.02,
 0.12,
 0.0,
 0.0,
 -0.0,
 0.0,
 0.0,
 0.0,
 -0.0,
 0.0,
 0.0,
 0.07,
 -0.06999999999999995,
 -0.0,
 -0.07,
 0.06999999999999995,
 0.0,
 0.02,
 -0.020000000000000018,
 -0.0,
 -0.02,
 0.020000000000000018]

### With all the building blocks above, we now can construct the CC test:

In [19]:
def T_n_cvxpy(theta, b_theta, moments = moments):
    mu = cp.Variable(len(moments))
    moments = np.array(moments)
    b_theta = np.array(b_theta)
    
    objective = cp.Minimize(cp.quad_form(moments - mu, Sigma_hat_inverse)) # we don't have to *n
    #objective = cp.Minimize(0) # this is to test whether feasible region exist
    constraints = [A @ mu <= b_theta]

    # Initialize tn and mu_hat to None
    tn, mu_hat = None, None

    
    prob = cp.Problem(objective, constraints)
    prob.solve(solver=cp.ECOS)

    # If the problem is solved optimally, break from the loop
    if prob.status in ["optimal", "optimal_inaccurate"]:
        tn, mu_hat = prob.value, mu.value

    if tn is None or mu_hat is None:
        print("Problem not solved:", prob.status)

    return tn, mu_hat

    
def compute_J_hat(A, mu_hat, b_theta):
    return np.sum(np.isclose(np.dot(A, mu_hat), b_theta, atol=1e-6))

def Phi_cvxpy(theta, alpha=0.05):
    b_theta = generate_b(theta,non_zero_moments)
    #print('len',len(b_theta))
    tn, mu_hat = T_n_cvxpy(theta, b_theta)
    if tn is None or mu_hat is None:
        return 1
    else:   
        print("Tn:", tn)
        J_hat = compute_J_hat(A, mu_hat, b_theta)
        print(J_hat)
        K = chi2.ppf(1 - alpha, J_hat)
        print("cv:", K)

        # Return 1 (reject) if condition is met, otherwise return 0
        return 1 if tn > max(K, 1e-8) else 0

In [20]:
# I manually found this theta^0 which is not rejected

theta_example = [0.26, 0.27, 1.5, 0.8, -1.89, -2.35, 1, 2.3, 1]

start_time = time.time()

result = Phi_cvxpy(theta_example)
print(result)

end_time = time.time()

# Print the elapsed time
print(f"The function took {end_time - start_time} seconds to complete.")

Tn: 29.547454398195903
26
cv: 38.885138659830055
0
The function took 0.06673526763916016 seconds to complete.


In [21]:
def objective(theta_partial, fixed_index, fixed_value, alpha=0.05):
    theta = np.insert(theta_partial, fixed_index, fixed_value)
    tn, mu_hat = T_n_cvxpy(theta, generate_b(theta, non_zero_moments), moments)
    
    if tn is None:
        return float('inf')  # this will discourage the optimizer from going here

    J_hat = compute_J_hat(A, mu_hat, generate_b(theta, non_zero_moments))
    cv_theta = chi2.ppf(1 - alpha, J_hat)
    return tn - cv_theta


def closest_theta(thetas, target_value, index):
    """Find the theta in thetas where theta[index] is closest to target_value."""
    differences = [abs(theta[index] - target_value) for theta in thetas]
    return thetas[np.argmin(differences)]

In [22]:
# this function estimates confidence intervals using only strategic selection

def find_confidence_interval_with_strategic_starts(theta_0, k, param_bounds, sim_bounds, stepsize, alpha=0.05):
    def reduced_guess(full_theta, index):
        """Return a guess vector with the element at the given index removed."""
        return np.delete(full_theta, index)

    accepted_thetas = [theta_0]  # Initial list of accepted thetas
    num_parameters = len(theta_0)
    
    for index in range(num_parameters):
        # Get the lb and ub for this parameter from bounds
        param_lb, param_ub = param_bounds[index]
        
        # Loop over the possible values for this parameter
        theta_values = np.arange(param_lb, param_ub, stepsize)
        
        for theta_val in theta_values:
            # Get the closest theta to current theta_val for the current parameter
            closest_full_theta = closest_theta(accepted_thetas, theta_val, index)
            initial_guess_closest = reduced_guess(closest_full_theta, index)
            
            # default guess
            default_guess = reduced_guess(theta_0, index)
            # Generate k random starting values
            random_initial_guesses = []
            for _ in range(k): # we don't select seeds for this one!
                random_theta = []
                for j, (sim_lb, sim_ub) in enumerate(sim_bounds):
                    if j != index:  # Skip the parameter we're currently testing
                        random_theta.append(np.random.uniform(sim_lb, sim_ub))
                random_initial_guesses.append(random_theta)
            
            for initial_guess in [default_guess] + [initial_guess_closest] + random_initial_guesses:
                result = minimize(
                    objective, 
                    initial_guess,  # This initial_guess is already reduced-sized
                    args=(index, theta_val, alpha), 
                    bounds=[b for j, b in enumerate(param_bounds) if j != index]  # Exclude the bounds for the parameter we're testing
                )

                if result.fun < 0:  # If minimized value is less than 0
                    accepted_thetas.append(np.insert(result.x, index, theta_val))
                    print(f"theta[{index}] = {round(theta_val, 2)} is accepted with value:", round(result.fun, 2))
                    break
                else:
                    print(f"theta[{index}] = {round(theta_val, 2)} is rejected with value:", round(result.fun, 2))
    
    # Returning the range of accepted values for all theta elements
    bounds_accepted = [(min([theta[i] for theta in accepted_thetas]), 
                        max([theta[i] for theta in accepted_thetas])) for i in range(num_parameters)]
    
    return bounds_accepted



In [23]:
# this function estimates confidence intervals using both strategic selection and seed control

def find_confidence_interval_with_strategic_seeds_control(theta_0, k, param_bounds, sim_bounds, stepsize, alpha=0.05):
    
    
    def reduced_guess(full_theta, index):
        """Return a guess vector with the element at the given index removed."""
        return np.delete(full_theta, index)
    
    def seeds_control(obj_fcn_value_list, k):
        """
        Keep seeds corresponding to the k/2 best objective function values.
        Replace the other k/2 seeds with new random ones.
        """
        # Get indices of objective function values sorted in ascending order
        sorted_indices = np.argsort(obj_fcn_value_list)

        # Keep the best k/2 seeds
        best_seeds = [seeds[i] for i in sorted_indices[:k//2]]

        # Generate new random seeds for the worst k/2 objective values
        # we don't use seed for this random generation
        new_seeds = list(np.random.randint(0, 1e5, size=k//2))

        # Return combined list of seeds
        return best_seeds + new_seeds



    accepted_thetas = [theta_0]  
    num_parameters = len(theta_0)

    for index in range(num_parameters):
        
        # initial seeds generation
        np.random.seed(66) # we control this generation using a seed
        seeds = np.random.randint(0, 1e5, size=k)
        
        # grid search over theta[i] to obtain its lower and upper bound
        param_lb, param_ub = param_bounds[index]
        sim_lb, sim_ub = sim_bounds[index]
        theta_values = np.arange(param_lb, param_ub, stepsize)
            
        for theta_val in theta_values:
            
            acception_flag = False # a flag indicates if theta[i]_j is accepted
            
            # pick the closest theta from the accepted bin
            closest_full_theta = closest_theta(accepted_thetas, theta_val, index) 
            initial_guess_closest = reduced_guess(closest_full_theta, index)
            
            # use theta_0 as our default guess
            default_guess = reduced_guess(theta_0, index)

            # k random simulations as initial guesses for starting parameters
            
            initial_guesses = []
            for _ in range(k): # we don't select seeds for this one!
                random_theta = []
                for j, (sim_lb, sim_ub) in enumerate(sim_bounds):
                    if j != index:  # Skip the parameter we're currently testing
                        random_theta.append(np.random.RandomState(seeds[_]).uniform(sim_lb, sim_ub))
                initial_guesses.append(random_theta)
            
            obj_fcn_value_list = [] # store objective function value wrt initial guesses
            
            idc = 0 # this is for storing obj fcn value
            
            for initial_guess in [default_guess] + [initial_guess_closest] + initial_guesses:
                result = minimize(
                    objective, 
                    initial_guess,  # This initial_guess is already reduced-sized
                    args=(index, theta_val, alpha), 
                    bounds=[b for j, b in enumerate(param_bounds) if j != index]  # Exclude the bounds for the parameter we're testing
                )
                
                if idc >= 2:
                    obj_fcn_value_list.append(result.fun)
                    
                idc += 1
                
                if result.fun < 0:  # If theta[i]_j is accepted, store it in the accepted bin
                    accepted_thetas.append(np.insert(result.x, index, theta_val))
                    print(f"theta[{index}] = {round(theta_val, 2)} is accepted with value:", round(result.fun, 2))
                    acception_flag = True
                    break
                else: 
                    print(f"theta[{index}] = {round(theta_val, 2)} is rejected with value:", round(result.fun, 2))
                
            # if theta[i]_j is rejected for all starting parameters, strategically updata seeeds for next theta[i]_j+1
            if acception_flag == False:
                seeds = seeds_control(obj_fcn_value_list, k)


    # Returning the range of accepted values for all theta elements
    bounds_accepted = [(min([theta[i] for theta in accepted_thetas]), 
                        max([theta[i] for theta in accepted_thetas])) for i in range(num_parameters)]
    
    return bounds_accepted


In [24]:
# this will estimate confidence intervals with predetermined bounds
# the computational time is approximately 10-15 hours

# initial bin of accepted parameters
theta_0_value = [0.26, 0.27, 1.5, 0.8, -1.89, -2.35, 1, 2.3, 1]

# number of starting parameters
k_value = 30

# parameter space
param_bounds_value = [(-0.5, 1.2), (-1, 1), (1, 3), (-0.1, 1.5), (-3, -1), (-4, -1), (-0.1, 2), (1, 3.5), (0, 2)]  # Specify bounds for all elements of theta
sim_bounds_value = [(-0.05, 0.62), (-0.5, 0.7), (1, 2), (0.1, 0.9), (-2.1, -1.8), (-3, -2), (0.8,1.2), (1.5, 2.5), (0.8, 1.5)]
step_value = 0.01
alpha_value = 0.05

# uncomment these lines when estimate
#result = find_confidence_interval_with_strategic_seeds_control(theta_0_value, k_value, param_bounds_value, sim_bounds_value, step_value, alpha_value)
#result

In [25]:
# infeasible grid search


# def grid_search():
#     accepted_thetas = []  # to store the accepted theta values

#     # Create a grid for the first 8 dimensions
#     grid_1_8 = np.linspace(-10, 10, num=21)  # adjust num as needed for grid granularity

#     # Create a grid for the 9th dimension
#     grid_9 = np.linspace(0, 5, num=6)  # adjust num as needed for grid granularity

#     for t1 in grid_1_8:
#         for t2 in grid_1_8:
#             for t3 in grid_1_8:
#                 for t4 in grid_1_8:
#                     for t5 in grid_1_8:
#                         for t6 in grid_1_8:
#                             for t7 in grid_1_8:
#                                 for t8 in grid_1_8:
#                                     for t9 in grid_9:
#                                         theta = [t1, t2, t3, t4, t5, t6, t7, t8, t9]
#                                         if not Phi_cvxpy(theta, A, m, Sigma_hat_inverse, n, alpha):
#                                             accepted_thetas.append(theta)
#                                             print(theta)

#     return accepted_thetas

# accepted_values = grid_search()
# print(accepted_values)
